# First Growth Curves

In [1]:
import numpy as np 
import pandas as pd
import gcutils
import altair as alt
from altair_saver import save
import scipy.stats
import matplotlib.pyplot as plt
# Load custom plotting style and colors
colors, palette = gcutils.viz.matplotlib_style()
colors, palette = gcutils.viz.altair_style()

This notebook contains the analysis of my first few sets of bacterial growth curves and compares the results to the "gold-standard" growth curves provided by Jonas. 

## January 15 Growth Data
Jonas shared with me a dataset of bacterial growth curves obtained on a variety of well-defined carbon substrates. I tidied up the data so it's more easily readable, which we load and plot below.


In [2]:
# Load the data 
gold_std = pd.read_csv('../../data/carbon_source_growth_data.csv')
data = pd.read_csv('../../data/2021-01-15_glucose_growth.csv')

# Convert my minute tracking to hours
data['elapsed_time_hr'] = data['elapsed_time_min'].values / 60
data.drop(columns=['elapsed_time_min'], inplace=True)
data.rename(columns={'od_600nm':'od_600'}, inplace=True)

# Drop points below 0.4 and rescale time
data = data[data['od_600'] >= 0.04]
data['elapsed_time_hr'] -= data['elapsed_time_hr'].min()

# Add identifiers
data['source'] = 'Griffin'
gold_std['source'] = 'Jonas'

# Restrict to glucose
gold_std = gold_std[gold_std['carbon_source']=='glucose']

# Merge
merged = pd.concat([data, gold_std], sort=False)

# Set up the plot base.
base = alt.Chart(merged).encode(
        x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
        y=alt.Y(field='od_600', type='quantitative', title='optical density [a.u.]',
               scale=alt.Scale(type='log')),
        shape=alt.Shape(field='source', type='nominal'),
        color=alt.Color(field='source', type='nominal', title='data source')
)
points = base.mark_point(size=70)
points

alt.Chart(...)

This doesn't look *too* bad, but my data (in black) definitely has a different slope than the gold-standard. On a semi-log scaling, that's not good. Let's do a simple linear regression of the log transform data to see what the slopes are

In [3]:
# Do a stupid fit for now. 
griffin = merged[merged['source']=='Griffin']
jonas = merged[merged['source']=='Jonas']
griffin_slope, griffin_int, _, _, griffin_se = scipy.stats.linregress(griffin['elapsed_time_hr'].values, np.log(griffin['od_600'].values))
jonas_slope, jonas_int, _, _, jonas_se = scipy.stats.linregress(jonas['elapsed_time_hr'].values, np.log(jonas['od_600'].values))


# Compute the fits and make a dataframe
time_range = np.linspace(0, 3.2, 200)
g_fit = griffin_int + griffin_slope * time_range
j_fit = jonas_int + jonas_slope * time_range
griffin_df = pd.DataFrame(np.array([time_range, np.exp(g_fit)]).T, columns=['time', 'y'])
griffin_df['source'] = 'Griffin'
jonas_df = pd.DataFrame(np.array([time_range, np.exp(j_fit)]).T, columns=['time', 'y'])
jonas_df['source'] = 'Jonas'
fit_merge = pd.concat([griffin_df, jonas_df], sort=False)

# Set up the figure
fit_base = alt.Chart(fit_merge).encode(
        x=alt.X(field='time', type='quantitative', title='elapsed time [hr]'),
        y=alt.Y(field='y', type='quantitative', title='optical density [a.u.]',
                scale=alt.Scale(type='log')),
        color=alt.Color(field='source', type='nominal'))

print(f"""
Growth rate (Griffin's data): λ = {griffin_slope:0.2f} ± {griffin_se:0.2f} hr^-1,
Growth rate (Jonas' data): λ = {jonas_slope:0.2f} ± {jonas_se:0.2f} hr^-1
""")
fit_curves = fit_base.mark_line(size=2, opacity=0.5)
fit_curves + points


Growth rate (Griffin's data): λ = 0.69 ± 0.02 hr^-1,
Growth rate (Jonas' data): λ = 0.84 ± 0.02 hr^-1



alt.LayerChart(...)

The fits look fair, let's see what the growth rate is

In [4]:
print(f"""
Growth rate (Griffin's data): λ = {griffin_slope:0.2f} ± {griffin_se:0.2f} hr^-1,
Growth rate (Jonas' data): λ = {jonas_slope:0.2f} ± {jonas_se:0.2f} hr^-1
""")


Growth rate (Griffin's data): λ = 0.69 ± 0.02 hr^-1,
Growth rate (Jonas' data): λ = 0.84 ± 0.02 hr^-1



There's a substantial difference in the growth rate. According to Jonas, growth on minimal media supplemented with glucose should be closer to 0.9 or so. This may have something to do with the preculture or the actual growth media. Next week, I will try to rerun the experiment and get better growth curves. 

## January 18 Growth Data 

In [5]:
# Load the days growth data
jan18_data = pd.read_csv('../../data/2021-01-18_glucose_growth.csv')

# Specify jonas' carbon source
gold_std['carbon_source'] = 'glucose_jonas'

# Add a source info, convert elapsed time to hr, and merge with other data
jan18_data['elapsed_time_hr'] = jan18_data['elapsed_time'].values / 60
jan18_data.rename(columns={'od_600nm':'od_600'}, inplace=True)
jan18_data['source'] = 'Griffin'

# Merge
merged = pd.concat([gold_std, jan18_data], sort=False)
base = alt.Chart(merged).encode(
        x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
        y=alt.Y(field='od_600', type='quantitative', title='optical density [a.u.]',
               scale=alt.Scale(type='log')),
        color = alt.Color(field='source', type='nominal', title='data source'),
        shape=alt.Shape(field='carbon_source', type='nominal', title='glucose stock'))

points = base.mark_point()
points

alt.Chart(...)

In [6]:
# Compute the simple fit. 
time = np.linspace(0, 3.5, 100)
fit_stats = pd.DataFrame([])
fit_dfs = []
for g, d in merged.groupby(['source', 'carbon_source']):
    slope, inter, _, _, se = scipy.stats.linregress(d['elapsed_time_hr'].values, 
                                                    np.log(d['od_600'].values))
    fit_stats = fit_stats.append({'source':g[0], 'carbon_source':g[1], 'intercept': inter, 'slope':slope, 'sem':se},
                            ignore_index=True)
    
    # Compute the fit over elapsed time
    fit = inter + slope * time
    _df = pd.DataFrame(np.array([time, np.exp(fit)]).T, columns=['elapsed_time_hr', 'od_600'])
    _df['source'] = g[0]
    _df['carbon_source'] = g[1]
    fit_dfs.append(_df)
    print(f"{g[0]} using  '{g[1]}' glucose stock: λ = {slope:0.2f} ± {se:0.2f} hr^-1")
fit_df = pd.concat(fit_dfs)

optimal = pd.DataFrame(np.array([time, np.exp(-3.2 + 0.9 * time)]).T, columns=['elapsed_time_hr', 'od_600'])

optimal_plot = alt.Chart(optimal).encode(
                x=alt.X(field='elapsed_time_hr', type='quantitative', title=''),
                y=alt.Y(field='od_600', type='quantitative', title='',
                        scale=alt.Scale(type='log'))).mark_line(color=colors['primary_red'])
    
fit_base = alt.Chart(fit_df).encode(
            x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
            y=alt.Y(field='od_600', type='quantitative', title='optical density [a.u.]',
                   scale=alt.Scale(type='log')),
            strokeDash='carbon_source:Q',
            color=alt.Color(field='source', type='nominal'))

fits = fit_base.mark_line()
jan18_plot = fits + points
optimal_plot + jan18_plot

Griffin using  'glucose' glucose stock: λ = 0.72 ± 0.01 hr^-1
Griffin using  'glucose_jonas' glucose stock: λ = 0.72 ± 0.01 hr^-1
Jonas using  'glucose_jonas' glucose stock: λ = 0.84 ± 0.02 hr^-1


alt.LayerChart(...)

## January 20

In [7]:
# Load and clean the data
jan20 = pd.read_csv('../../data/2021-01-20_r1_GE046_GE047_GC001_growth.csv')

#Drop the first time point because it is weird
jan20 = jan20[jan20['elapsed_time_min'] > 0]
jan20['elapsed_time_min'] -= jan20['elapsed_time_min'].min()
jan20['elapsed_time_hr'] = jan20['elapsed_time_min'].values / 60

# Plot!
base = alt.Chart(jan20).encode(
        x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
        y=alt.Y(field='od_600nm', type='quantitative', title='optical density [a.u.]',
                scale=alt.Scale(type='log')),
        color=alt.Color(field='strain', type='nominal', title='strain'))
#         shape=alt.Shape(field='strain', type='nominal', title='strain'))

jan20_point = base.mark_point(size=80)


# SHow the optimal one
optimal = pd.DataFrame(np.array([time, np.exp(-3 + 0.9 * time)]).T, columns=['elapsed_time_hr', 'od_600'])

# optimal_plot = alt.Chart(optimal).encode(
#                 x=alt.X(field='elapsed_time_hr', type='quantitative', title=''),
#                 y=alt.Y(field='od_600', type='quantitative', title='',
#                         scale=alt.Scale(type='log'))).mark_line(color=colors['primary_red'])
    
    
    
# Compute the fits and plot on top.
time = np.linspace(0, 3.5, 100)
fit_stats = pd.DataFrame([])
fit_dfs = []
for g, d in jan20.groupby(['strain']):
    slope, inter, _, _, se = scipy.stats.linregress(d['elapsed_time_hr'].values, 
                                                    np.log(d['od_600nm'].values))
    fit_stats = fit_stats.append({'strain':g, 'intercept': inter, 'slope':slope, 'sem':se},
                            ignore_index=True)
              
    # Compute the fit over elapsed time
    fit = inter + slope * time
    _df = pd.DataFrame(np.array([time, np.exp(fit)]).T, columns=['elapsed_time_hr', 'od_600'])
    _df['strain'] = g
    _df['growth_rate'] = f'{slope:0.2f} ± {se:0.2f}'
    fit_dfs.append(_df)
    print(f"Strain {g}: λ = {slope:0.2f} ± {se:0.2f} hr^-1")
fit_df = pd.concat(fit_dfs)


fit_base = alt.Chart(fit_df).encode(
            x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
            y=alt.Y(field='od_600', type='quantitative', title='optical density [a.u.]',
                    scale=alt.Scale(type='log')),
            color=alt.Color(field='growth_rate', type='nominal',
                            scale=alt.Scale(domain=fit_df['growth_rate'].unique(),
                                            range=palette[:3]),
                            title='inferred growth rate [hr\u207B\u00b9]'))

fit_plot = fit_base.mark_line(size=2, opacity=0.5)
layer = alt.layer(jan20_point, fit_plot).resolve_scale(color='independent')
layer
# layer.save('/Users/gchure/Desktop/2020-01-20_strain_growth.png')

Strain GC001: λ = 0.73 ± 0.01 hr^-1
Strain GE046: λ = 0.76 ± 0.01 hr^-1
Strain GE047: λ = 0.49 ± 0.01 hr^-1


alt.LayerChart(...)

In [8]:
# Load and clean the data
jan20_r2 = pd.read_csv('../../data/2021-01-20_r2_GE046_GC001_growth.csv')

#Drop the first time point because it is weird
jan20_r2 = jan20_r2[(jan20_r2['od_600nm'] >= 0.04)]
jan20_r2['elapsed_time_min'] -= jan20_r2['elapsed_time_min'].min()
jan20_r2['elapsed_time_hr'] = jan20_r2['elapsed_time_min'].values / 60

# Plot!
base = alt.Chart(jan20_r2).encode(
        x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
        y=alt.Y(field='od_600nm', type='quantitative', title='optical density [a.u.]',
                scale=alt.Scale(type='log')),
        color=alt.Color(field='strain', type='nominal', title='strain'))
#         shape=alt.Shape(field='strain', type='nominal', title='strain'))

jan20_point = base.mark_point(size=80)
    
    
# Compute the fits and plot on top.
time = np.linspace(0, 3.5, 300)
fit_stats = pd.DataFrame([])
fit_dfs = []
for g, d in jan20_r2.groupby(['strain']):
    slope, inter, _, _, se = scipy.stats.linregress(d['elapsed_time_hr'].values, 
                                                    np.log(d['od_600nm'].values))
    fit_stats = fit_stats.append({'strain':g, 'intercept': inter, 'slope':slope, 'sem':se},
                            ignore_index=True)
              
    # Compute the fit over elapsed time
    fit = inter + slope * time
    _df = pd.DataFrame(np.array([time, np.exp(fit)]).T, columns=['elapsed_time_hr', 'od_600'])
    _df['strain'] = g
    _df['growth_rate'] = f'{slope:0.2f} ± {se:0.2f}'
    fit_dfs.append(_df)
    print(f"Strain {g}: λ = {slope:0.2f} ± {se:0.2f} hr^-1")
fit_df = pd.concat(fit_dfs)


fit_base = alt.Chart(fit_df).encode(
            x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
            y=alt.Y(field='od_600', type='quantitative', title='optical density [a.u.]',
                    scale=alt.Scale(type='log')),
            color=alt.Color(field='growth_rate', type='nominal',
                            scale=alt.Scale(domain=fit_df['growth_rate'].unique(),
                                            range=palette[:3]),
                            title='inferred growth rate [hr\u207B\u00b9]'))

fit_plot = fit_base.mark_line(size=2, opacity=0.5)
layer = alt.layer(jan20_point, fit_plot).resolve_scale(color='independent')
layer
save(layer, './test.png')

Strain GC001: λ = 0.80 ± 0.03 hr^-1
Strain GE046: λ = 0.81 ± 0.02 hr^-1


## January 21

In [9]:
# Load and clean the data
jan21 = pd.read_csv('../../data/2021-01-21_tube_size_growth.csv')
jan21 = jan21[(jan21['elapsed_time_min'] >=0) &
               (jan21['elapsed_time_min'] <= 190) & (jan21['od_600nm'] >= 0.04) & (jan21['od_600nm'] <= 0.4)]
jan21['elapsed_time_min'] -= jan21['elapsed_time_min'].min()
jan21['elapsed_time_hr'] = jan21['elapsed_time_min'].values / 60

# Plot!
base = alt.Chart(jan21).encode(
        x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
        y=alt.Y(field='od_600nm', type='quantitative', title='optical density [a.u.]',
                scale=alt.Scale(type='log')),
        color=alt.Color(field='tube_diam_mm', type='nominal', title='tube diameter [mm]'),
        shape=alt.Shape(field='preculture_id', type='nominal', title='preculture ID'))

jan21_point = base.mark_point(size=80)

    
# Compute the fits and plot on top.
time = np.linspace(0, 2.5, 100)
fit_stats = pd.DataFrame([])
fit_dfs = []
for g, d in jan21.groupby(['tube_diam_mm', 'preculture_id']):
    slope, inter, _, _, se = scipy.stats.linregress(d['elapsed_time_hr'].values, 
                                                    np.log(d['od_600nm'].values))
    fit_stats = fit_stats.append({'strain':g, 'intercept': inter, 'slope':slope, 'sem':se},
                            ignore_index=True)
              
    # Compute the fit over elapsed time
    fit = inter + slope * time
    _df = pd.DataFrame(np.array([time, np.exp(fit)]).T, columns=['elapsed_time_hr', 'od_600'])
    _df['tube_diam_mm'] = g[0]
    _df['preculture_id'] = g[1]
    _df['growth_rate'] = f'{slope:0.2f} ± {se:0.2f}'
    fit_dfs.append(_df)
    print(f"Tube diameter {g[0]} mm; preculture {g[1]}: λ = {slope:0.2f} ± {se:0.2f} hr^-1")
fit_df = pd.concat(fit_dfs)


fit_base = alt.Chart(fit_df).encode(
            x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
            y=alt.Y(field='od_600', type='quantitative', title='optical density [a.u.]',
                    scale=alt.Scale(type='log')),
            color=alt.Color(field='tube_diam_mm', type='nominal', legend=None),
#                             scale=alt.Scale(domain=fit_df['growth_rate'].unique(),
#                                             range=palette[:4]),
#                             title='inferred growth rate [hr\u207B\u00b9]'),
            strokeDash=alt.StrokeDash(field='preculture_id', type='nominal', title='preculture ID'))

fit_plot = fit_base.mark_line(size=2, opacity=0.5)
layer = alt.layer(jan21_point, fit_plot).resolve_scale(color='independent')
layer

Tube diameter 20 mm; preculture 1: λ = 0.75 ± 0.01 hr^-1
Tube diameter 20 mm; preculture 2: λ = 0.75 ± 0.01 hr^-1
Tube diameter 25 mm; preculture 1: λ = 0.77 ± 0.02 hr^-1
Tube diameter 25 mm; preculture 2: λ = 0.76 ± 0.01 hr^-1


alt.LayerChart(...)

## January 22

In [10]:
# Load and clean the data
jan22 = pd.read_csv('../../data/2021-01-22_RNC_GNC_JNC_media_growth.csv')
jan22 = jan22[(jan22['od_600nm'] >= 0.04) & (jan22['od_600nm'] <= 0.4) & (jan22['note']=='ok')]
jan22['elapsed_time_min'] -= jan22['elapsed_time_min'].min()
jan22['elapsed_time_hr'] = jan22['elapsed_time_min'].values / 60

# Plot!
base = alt.Chart(jan22).encode(
        x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
        y=alt.Y(field='od_600nm', type='quantitative', title='optical density [a.u.]',
                scale=alt.Scale(type='log')),
        color=alt.Color(field='media_source', type='nominal', title='Medium Source'))
jan22_point = base.mark_point(size=80, opacity=0.5)

    
# Compute the fits and plot on top.
time = np.linspace(0, 2.5, 100)
fit_stats = pd.DataFrame([])
fit_dfs = []
for g, d in jan22.groupby(['media_source']):
    slope, inter, _, _, se = scipy.stats.linregress(d['elapsed_time_hr'].values, 
                                                    np.log(d['od_600nm'].values))
    fit_stats = fit_stats.append({'media_source':g, 'intercept': inter, 'slope':slope, 'sem':se},
                            ignore_index=True)
              
    # Compute the fit over elapsed time
    fit = inter + slope * time
    _df = pd.DataFrame(np.array([time, np.exp(fit)]).T, columns=['elapsed_time_hr', 'od_600'])
    _df['media_source'] = g
    _df['growth_rate'] = f'{slope:0.2f} ± {se:0.2f}'
    fit_dfs.append(_df)
    print(f"Media source '{g}': λ = {slope:0.2f} ± {se:0.2f} hr^-1")
fit_df = pd.concat(fit_dfs)


fit_base = alt.Chart(fit_df).encode(
            x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
            y=alt.Y(field='od_600', type='quantitative', title='optical density [a.u.]',
                    scale=alt.Scale(type='log')),
            color=alt.Color(field='media_source', type='nominal', legend=None))
#                             scale=alt.Scale(domain=fit_df['growth_rate'].unique(),
#                                             range=palette[:4]),
#                             title='inferred growth rate [hr\u207B\u00b9]'),
        
fit_plot = fit_base.mark_line(size=2, opacity=0.5)
layer = alt.layer(jan22_point, fit_plot).resolve_scale(color='independent')
layer.interactive()

Media source 'Griffin': λ = 0.73 ± 0.01 hr^-1
Media source 'Griffin (Jonas' Ammonium)': λ = 0.71 ± 0.02 hr^-1
Media source 'Roshali': λ = 0.85 ± 0.01 hr^-1


alt.LayerChart(...)

## January 26


In [11]:
# Load and clean the data
jan26 = pd.read_csv('../../data/2021-01-26_media_dissection_growth.csv')

# Drop Roshali's since it's weird
jan26 = jan26[(jan26['od_600nm'] >= 0.04) & (jan26['od_600nm'] <= 0.4) & 
              (jan26['media'] != 'Roshali')]
jan26['elapsed_time_min'] -= jan26['elapsed_time_min'].min()
jan26['elapsed_time_hr'] = jan26['elapsed_time_min'].values / 60


# Plot!
base = alt.Chart(jan26).encode(
        x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
        y=alt.Y(field='od_600nm', type='quantitative', title='optical density [a.u.]',
                scale=alt.Scale(type='log')),
        color=alt.Color(field='media', type='nominal', title='Medium Source/Composition'))
jan26_point = base.mark_point(size=80)

    
#Compute the fits and plot on top.
time = np.linspace(0, 3, 100)
fit_stats = pd.DataFrame([])
fit_dfs = []
for g, d in jan26.groupby(['media']):
    slope, inter, _, _, se = scipy.stats.linregress(d['elapsed_time_hr'].values, 
                                                    np.log(d['od_600nm'].values))
    fit_stats = fit_stats.append({'media_source':g, 'intercept': inter, 'slope':slope, 'sem':se},
                            ignore_index=True)
              
    # Compute the fit over elapsed time
    fit = inter + slope * time
    _df = pd.DataFrame(np.array([time, np.exp(fit)]).T, columns=['elapsed_time_hr', 'od_600'])
    _df['media'] = g
    _df['growth_rate'] = f'{slope:0.2f} ± {se:0.2f}'
    fit_dfs.append(_df)
    print(f"Media source '{g}': λ = {slope:0.2f} ± {se:0.2f} hr^-1")
fit_df = pd.concat(fit_dfs)


fit_base = alt.Chart(fit_df).encode(
            x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
            y=alt.Y(field='od_600', type='quantitative', title='optical density [a.u.]',
                    scale=alt.Scale(type='log')),
            color=alt.Color(field='media', type='nominal', legend=None))
        
fit_plot = fit_base.mark_line(size=2, opacity=0.5)
layer = alt.layer(jan26_point, fit_plot).resolve_scale(color='independent')
layer.interactive()

Media source 'Griffin': λ = 0.79 ± 0.01 hr^-1
Media source 'Griffin + 2x Mg': λ = 0.85 ± 0.01 hr^-1
Media source 'Roshali Glucose': λ = 0.77 ± 0.01 hr^-1
Media source 'Roshali H2O': λ = 0.78 ± 0.01 hr^-1
Media source 'Roshali N-C-': λ = 0.87 ± 0.01 hr^-1
Media source 'Roshali NH4': λ = 0.78 ± 0.01 hr^-1


alt.LayerChart(...)

## January 28

In [12]:
# Load and clean the data
jan28 = pd.read_csv('../../data/2021-01-28_Mg_media_growth.csv')
jan28 = jan28[jan28['media']!= 'LH Spec Blank']
jan28 = jan28[(jan28['od_600nm'] >= 0.04) & (jan28['od_600nm'] <= 0.4)]  
jan28['elapsed_time_min'] -= jan28['elapsed_time_min'].min()
jan28['elapsed_time_hr'] = jan28['elapsed_time_min'].values / 60


# Plot!
base = alt.Chart(jan28).encode(
        x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
        y=alt.Y(field='od_600nm', type='quantitative', title='optical density [a.u.]',
                scale=alt.Scale(type='log')),
        color=alt.Color(field='media', type='nominal', title='Medium Source/Composition'))
jan28_point = base.mark_point(size=80)

    
#Compute the fits and plot on top.
time = np.linspace(0, 3, 100)
fit_stats = pd.DataFrame([])
fit_dfs = []
for g, d in jan28.groupby(['media']):
    slope, inter, _, _, se = scipy.stats.linregress(d['elapsed_time_hr'].values, 
                                                    np.log(d['od_600nm'].values))
    fit_stats = fit_stats.append({'media_source':g, 'intercept': inter, 'slope':slope, 'sem':se},
                            ignore_index=True)
              
    # Compute the fit over elapsed time
    fit = inter + slope * time
    _df = pd.DataFrame(np.array([time, np.exp(fit)]).T, columns=['elapsed_time_hr', 'od_600'])
    _df['media'] = g
    _df['growth_rate'] = f'{slope:0.2f} ± {se:0.2f}'
    fit_dfs.append(_df)
    print(f"Media source '{g}': λ = {slope:0.2f} ± {se:0.2f} hr^-1")
fit_df = pd.concat(fit_dfs)


fit_base = alt.Chart(fit_df).encode(
            x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
            y=alt.Y(field='od_600', type='quantitative', title='optical density [a.u.]',
                    scale=alt.Scale(type='log')),
            color=alt.Color(field='media', type='nominal', legend=None))
        
fit_plot = fit_base.mark_line(size=2, opacity=0.5)
layer = alt.layer(jan28_point, fit_plot).resolve_scale(color='independent')
layer.interactive()

Media source 'Griffin N-C-': λ = 0.80 ± 0.02 hr^-1
Media source 'Griffin N-C-+ Mg': λ = 0.84 ± 0.01 hr^-1
Media source 'Roshali N-C-': λ = 0.91 ± 0.02 hr^-1
Media source 'Roshali N-C- + Mg': λ = 0.94 ± 0.01 hr^-1


alt.LayerChart(...)

#  January 29

In [13]:
# Load and clean the data
jan29 = pd.read_csv('../../data/2021-01-29_MOPS_NC_media_growth.csv')
jan29 = jan29[(jan29['od_600nm'] >= 0.04) & (jan29['od_600nm'] <= 0.4)]  
dfs = []
for g, d in jan29.groupby(['media']):
    d['elapsed_time_min'] -= d['elapsed_time_min'].min()
    d['elapsed_time_hr'] = d['elapsed_time_min'].values / 60
    dfs.append(d)
jan29 = pd.concat(dfs, sort=False)


# Plot!
base = alt.Chart(jan29).encode(
        x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
        y=alt.Y(field='od_600nm', type='quantitative', title='optical density [a.u.]',
                scale=alt.Scale(type='log')),
        color=alt.Color(field='media', type='nominal', title='Medium'))
jan29_point = base.mark_point(size=80)

    
#Compute the fits and plot on top.
time = np.linspace(0, 3, 100)
fit_stats = pd.DataFrame([])
fit_dfs = []
for g, d in jan29.groupby(['media']):
    slope, inter, _, _, se = scipy.stats.linregress(d['elapsed_time_hr'].values, 
                                                    np.log(d['od_600nm'].values))
    fit_stats = fit_stats.append({'media_source':g, 'intercept': inter, 'slope':slope, 'sem':se},
                            ignore_index=True)
              
    # Compute the fit over elapsed time
    fit = inter + slope * time
    _df = pd.DataFrame(np.array([time, np.exp(fit)]).T, columns=['elapsed_time_hr', 'od_600'])
    _df['media'] = g
    _df['growth_rate'] = f'{slope:0.2f} ± {se:0.2f}'
    fit_dfs.append(_df)
    print(f"Media source '{g}': λ = {slope:0.2f} ± {se:0.2f} hr^-1")
fit_df = pd.concat(fit_dfs)


fit_base = alt.Chart(fit_df).encode(
            x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
            y=alt.Y(field='od_600', type='quantitative', title='optical density [a.u.]',
                    scale=alt.Scale(type='log')),
            color=alt.Color(field='media', type='nominal', legend=None))
        
fit_plot = fit_base.mark_line(size=2, opacity=0.5)
layer = alt.layer(jan29_point, fit_plot).resolve_scale(color='independent')
layer.interactive()

Media source 'MOPS': λ = 0.87 ± 0.01 hr^-1
Media source 'MOPS + Mg': λ = 0.95 ± 0.01 hr^-1
Media source 'N-C-': λ = 0.81 ± 0.01 hr^-1
Media source 'N-C- + Mg': λ = 0.83 ± 0.01 hr^-1
Media source 'N-C- + µ': λ = 0.93 ± 0.01 hr^-1
Media source 'N-C- + µ + Mg': λ = 0.91 ± 0.01 hr^-1


alt.LayerChart(...)

# February 1

In [14]:
# Load and clean the data
feb1 = pd.read_csv('../../data/2021-02-01_micronutrient_growth.csv')
feb1 = feb1[(feb1['od_600nm'] >= 0.04) & (feb1['od_600nm'] <= 0.4)]  

# Relabel the micronutrient identity for nice presentation
feb1.loc[feb1['micronutrient']==1, 'micronutrient'] = '+'
feb1.loc[feb1['micronutrient']==0, 'micronutrient'] = '-'

dfs = []
for g, d in feb1.groupby(['media', 'micronutrient']):
    d['elapsed_time_min'] -= d['elapsed_time_min'].min()
    d['elapsed_time_hr'] = d['elapsed_time_min'].values / 60
    dfs.append(d)
feb1 = pd.concat(dfs, sort=False)


# Plot!
base = alt.Chart(feb1).encode(
        x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
        y=alt.Y(field='od_600nm', type='quantitative', title='optical density [a.u.]',
                scale=alt.Scale(type='log')),
        color=alt.Color(field='media', type='nominal', title='Medium'),
        shape=alt.Shape(field='micronutrient', type='nominal', title='Micronutrients'))
feb1_point = base.mark_point(size=80)

    
#Compute the fits and plot on top.
time = np.linspace(0, 2.2, 100)
fit_stats = pd.DataFrame([])
fit_dfs = []
for g, d in feb1.groupby(['media','micronutrient']):
    slope, inter, _, _, se = scipy.stats.linregress(d['elapsed_time_hr'].values, 
                                                    np.log(d['od_600nm'].values))
    fit_stats = fit_stats.append({'media_source':g, 'intercept': inter, 'slope':slope, 'sem':se},
                            ignore_index=True)
              
    # Compute the fit over elapsed time
    fit = inter + slope * time
    _df = pd.DataFrame(np.array([time, np.exp(fit)]).T, columns=['elapsed_time_hr', 'od_600'])
    _df['media'] = g[0]
    _df['growth_rate'] = f'{slope:0.2f} ± {se:0.2f}'
    _df['micronutrient'] = g[1]
    fit_dfs.append(_df)
    print(f"{g[0]} {g[1]} micronutrients : λ = {slope:0.2f} ± {se:0.2f} hr^-1")
fit_df = pd.concat(fit_dfs)


fit_base = alt.Chart(fit_df).encode(
            x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
            y=alt.Y(field='od_600', type='quantitative', title='optical density [a.u.]',
                    scale=alt.Scale(type='log')),
            color=alt.Color(field='media', type='nominal', legend=None),
            strokeDash=alt.StrokeDash(field='micronutrient', type='nominal', legend=None))
        
fit_plot = fit_base.mark_line(size=2, opacity=0.5)
layer = alt.layer(feb1_point, fit_plot).resolve_scale(color='independent')
layer.interactive()

Griffin + micronutrients : λ = 0.95 ± 0.02 hr^-1
Griffin - micronutrients : λ = 0.84 ± 0.01 hr^-1
Jonas + micronutrients : λ = 0.96 ± 0.01 hr^-1
Jonas - micronutrients : λ = 0.82 ± 0.02 hr^-1
Richa + micronutrients : λ = 0.93 ± 0.00 hr^-1
Richa - micronutrients : λ = 0.69 ± 0.02 hr^-1
Roshali + micronutrients : λ = 0.94 ± 0.03 hr^-1
Roshali - micronutrients : λ = 0.86 ± 0.01 hr^-1


alt.LayerChart(...)

In [15]:
sub_dfs = []
for g, d in feb1.groupby(['media', 'micronutrient']):
    d.sort_values(by='elapsed_time_hr', inplace=True)
    d['od_sub'] = d['od_600nm'].values / d['od_600nm'].values[0]
    sub_dfs.append(d)
sub_df = pd.concat(sub_dfs, sort=False)

sub_fit_dfs = []
for g, d in fit_df.groupby(['media', 'micronutrient']):
    d.sort_values(by='elapsed_time_hr', inplace=True)
    d['od_sub'] = d['od_600'].values / d['od_600'].values[0]
    sub_fit_dfs.append(d)
sub_fit_df = pd.concat(sub_fit_dfs, sort=False)


# Plot!
base = alt.Chart(sub_df).encode(
        x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
        y=alt.Y(field='od_sub', type='quantitative', title='relative optical density',
                scale=alt.Scale(type='log')),
        color=alt.Color(field='media', type='nominal', title='Medium'),
        shape=alt.Shape(field='micronutrient', type='nominal', title='Micronutrients'))
feb1_point = base.mark_point(size=90, opacity=0.5)


fit_base = alt.Chart(sub_fit_df).encode(
            x=alt.X(field='elapsed_time_hr', type='quantitative', title='elapsed time [hr]'),
            y=alt.Y(field='od_sub', type='quantitative', title='relative optical density',
                    scale=alt.Scale(type='log')),
            color=alt.Color(field='media', type='nominal', legend=None),
            strokeDash=alt.StrokeDash(field='micronutrient', type='nominal', legend=None))
        
fit_plot = fit_base.mark_line(size=2, opacity=0.5)
(fit_plot+ feb1_point).interactive()

alt.LayerChart(...)

In [34]:
fit_df

,elapsed_time_hr,od_600,media,growth_rate,micronutrient
0,0.000000,0.061151,Griffin,0.95 ± 0.02,+
1,0.030303,0.062929,Griffin,0.95 ± 0.02,+
2,0.060606,0.064759,Griffin,0.95 ± 0.02,+
3,0.090909,0.066642,Griffin,0.95 ± 0.02,+
4,0.121212,0.068580,Griffin,0.95 ± 0.02,+
...,...,...,...,...,...
95,2.878788,0.477406,Roshali,0.85 ± 0.02,-
96,2.909091,0.489934,Roshali,0.85 ± 0.02,-
97,2.939394,0.502790,Roshali,0.85 ± 0.02,-
98,2.969697,0.515984,Roshali,0.85 ± 0.02,-
